In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
import gc

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [4]:
def means_for_teams(data):
    g_data = data.groupby(['groupId']).mean()
    print(g_data.shape)

    for i, groupId in zip(data.index, data['groupId']):
        data.iloc[i][g_data.columns] = g_data.loc[groupId]
    return data

# features for duo or squad mode, these features are valued 0 in solo mode games
def drop_team_features(data):
    team_features = ['assists', 'DBNOs', 'revives', 'teamkills']
    return data.drop(team_features, axis=1)

def xy(data):
    X = data.drop('winPlacePerc', axis = 1).select_dtypes(['number'])
    y = data['winPlacePerc']
    return X, y

def fill_rankPoints(input_data, model = LinearRegression(), dropWinKillPoints=True):
    work_data = input_data.copy()
    work_data = pd.DataFrame(work_data.select_dtypes(['number']))

    #drop the other point features, since these will not be useful in predicting rankPoints
    work_data.drop(['winPoints', 'killPoints'], axis=1, inplace=True)

    #train the model with data where there are rankPoints
    train = pd.DataFrame(work_data.loc[data['rankPoints'] > 0, :])
    X_train = train.drop('rankPoints', axis=1)
    y_train = train['rankPoints']
    model.fit(X_train, y_train)

    #use model to predict missing rankPoints
    use = pd.DataFrame(work_data.loc[data['rankPoints'] <= 0, :])
    X_use = use.drop('rankPoints', axis=1)
    y_use = model.predict(X_use)

    #fill in the missing data
    work_data.loc[work_data['rankPoints'] <= 0, 'rankPoints'] = y_use

    return work_data


def print_top_feature_correlations_to_target_by_matchType(data, nrows):
    matchTypes = data['matchType'].value_counts().index.tolist()
    matchCounts = data['matchType'].value_counts().values.tolist()

    print("-----------------------------------------------------")
    print("Highest correlations to target feature BY matchType")
    print("Number of games:", nrows, "\n")
    for cnt, mt in zip(matchCounts, matchTypes):
        # for each matchtype
        # look at all the rows for that matchtype
        # and build a correlation matrix
        corr = data \
            .drop('winPlacePerc', axis=1) \
            .loc[data['matchType'] == mt] \
            .corrwith(data.loc[data['matchType'] == mt]['winPlacePerc'])
        corr = corr.sort_values(ascending=False)
        print("Type:", mt)
        print("Nr of games:", cnt)
        print(corr.head())
        print()


def scoreSets(dfs, random_state=1):
    for i, df in enumerate(dfs):
        lr = LinearRegression()
        x, y = xy(df)
        xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=random_state)
        lr.fit(xtrain, ytrain)
        print("Score for set nr: ", i+1, lr.score(xtest, ytest))

def divide_by_matchType_and_aggregate_by_groupId(data):
    matchTypes = data['matchType'].value_counts().index.tolist()
    squadMatches = []
    duoMatches = []
    soloMatches = []
    otherMatches = []
    for match in matchTypes:
        if 'squad' in match or 'flare' in match:
            squadMatches.append(match)
        elif 'duo' in match or 'crash' in match:
            duoMatches.append(match)
        elif 'solo' in match:
            soloMatches.append(match)
        else:
            otherMatches.append(match)

    squadSet = pd.DataFrame(data.loc[data['matchType'].isin(squadMatches),:])
    duoSet = pd.DataFrame(data.loc[data['matchType'].isin(duoMatches),:])
    soloSet = pd.DataFrame(data.loc[data['matchType'].isin(soloMatches),:])

    squadMeanSet = pd.DataFrame(squadSet.groupby('groupId').mean(), dtype='float16')
    duoMeanSet = pd.DataFrame(duoSet.groupby('groupId').mean(), dtype='float16')

    squadSets = [squadSet, squadMeanSet]
    duoSets = [duoSet, duoMeanSet]

    #print("Returning [[squadSet, squadMeanSet], [duoSet, duoMeanSet], soloSet]")
    return [squadSets, duoSets, soloSet]


In [5]:
# assign dtypes to minimize memory usage
dtypes = {
    'Id': 'object',
    'groupId': 'object',
    'matchId': 'object',
    'assists': 'uint8',
    'boosts': 'uint8',
    'damageDealt': 'float16',
    'DBNOs': 'uint8',
    'headshotKills': 'uint8',
    'heals': 'uint8',
    'killPlace': 'uint8',
    'killPoints': 'uint16',
    'kills': 'uint8',
    'killStreaks': 'uint8',
    'longestKill': 'float16',
    'maxPlace': 'uint8',
    'numGroups': 'uint8',
    'revives': 'uint8',
    'rideDistance': 'float16',
    'roadKills': 'uint8',
    'swimDistance': 'float16',
    'teamKills': 'uint8',
    'vehicleDestroys': 'uint8',
    'walkDistance': 'float16',
    'weaponsAcquired': 'uint8',
    'winPoints': 'uint8',
    'winPlacePerc': 'float16'
}

In [6]:
train_data = pd.read_csv("train_V2.csv", dtype=dtypes)
test_data = pd.read_csv("test_V2.csv", dtype=dtypes)

print(train_data.loc[:, train_data.isnull().any()].columns)
print(test_data.loc[:, test_data.isnull().any()].columns)

train_data[train_data['winPlacePerc'].isnull()]
train_data = train_data.dropna(subset=['winPlacePerc'])
train_data[train_data['winPlacePerc'].isnull()]

Index(['winPlacePerc'], dtype='object')
Index([], dtype='object')


,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc


In [7]:
# train_data - Training data with feature "winPlacePerc"
# test_data - Testing data without feature "winPlacePerc"
# models - models for solo, dua and squad prediction as list [model for solo, model for duo, model for squad]

def testPredictions(train_data, test_data, models):

    train = train_data
    test = test_data
   
    trainSets = divide_by_matchType_and_aggregate_by_groupId(train)
    testSets = divide_by_matchType_and_aggregate_by_groupId(test)
    
    all_predictions = pd.DataFrame()
    for train, test, model in zip(trainSets, testSets, models):
        if len(train) == 2:
            trainset = train[0]
            meanTrainSet = train[1] #only numeric values, the index is 'groupId'
            testset = test[0]
            X_test = test[1]

            X_train = meanTrainSet.drop('winPlacePerc', axis=1)
            y_train = meanTrainSet.winPlacePerc

            model.fit(X_train, y_train)

            yhat = model.predict(X_test)

            #now I will rejoin the prediction with the original row index
            yhat = pd.DataFrame(yhat, index=X_test.index,columns=['prediction'])
            testset = testset.join(yhat, on='groupId')
            yhat = testset.drop(testset.columns.difference(['prediction']), axis=1)

        else:
            X_train = train.select_dtypes(['number']).drop('winPlacePerc', axis=1)
            y_train = train.winPlacePerc
            X_test = pd.DataFrame(test.select_dtypes(['number']))

            model.fit(X_train, y_train)
            yhat = model.predict(X_test)

            X_test['prediction'] = yhat
            yhat = pd.DataFrame(X_test['prediction'], index=X_test.index, dtype='float32')

        all_predictions = all_predictions.append(yhat)

    all_predictions.sort_index(inplace=True)

    return all_predictions


In [9]:
solo_mod = RandomForestRegressor(n_estimators=120, max_features=0.5, min_samples_leaf=3, n_jobs=-1, verbose=2)
duo_mod = RandomForestRegressor(n_estimators=120, max_features=0.5, min_samples_leaf=3, n_jobs=-1, verbose=2)
squad_mod = RandomForestRegressor(n_estimators=120, max_features=0.5, min_samples_leaf=3, n_jobs=-1, verbose=2)

models = [solo_mod, duo_mod, squad_mod]

train_d = train_data
test_d = test_data
all_predictions = testPredictions(train_d, test_d, models)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 120building tree 2 of 120building tree 3 of 120


building tree 4 of 120
building tree 5 of 120
building tree 6 of 120
building tree 7 of 120
building tree 8 of 120
building tree 9 of 120
building tree 10 of 120
building tree 11 of 120
building tree 12 of 120
building tree 13 of 120
building tree 14 of 120
building tree 15 of 120
building tree 16 of 120
building tree 17 of 120
building tree 18 of 120
building tree 19 of 120
building tree 20 of 120
building tree 21 of 120
building tree 22 of 120
building tree 23 of 120
building tree 24 of 120
building tree 25 of 120
building tree 26 of 120
building tree 27 of 120
building tree 28 of 120
building tree 29 of 120
building tree 30 of 120
building tree 31 of 120
building tree 32 of 120
building tree 33 of 120
building tree 34 of 120
building tree 35 of 120
building tree 36 of 120


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.8min


building tree 37 of 120
building tree 38 of 120
building tree 39 of 120
building tree 40 of 120
building tree 41 of 120
building tree 42 of 120
building tree 43 of 120
building tree 44 of 120
building tree 45 of 120
building tree 46 of 120
building tree 47 of 120
building tree 48 of 120
building tree 49 of 120
building tree 50 of 120
building tree 51 of 120
building tree 52 of 120
building tree 53 of 120
building tree 54 of 120
building tree 55 of 120
building tree 56 of 120
building tree 57 of 120
building tree 58 of 120
building tree 59 of 120
building tree 60 of 120
building tree 61 of 120
building tree 62 of 120
building tree 63 of 120
building tree 64 of 120
building tree 65 of 120
building tree 66 of 120
building tree 67 of 120
building tree 68 of 120
building tree 69 of 120
building tree 70 of 120
building tree 71 of 120
building tree 72 of 120
building tree 73 of 120
building tree 74 of 120
building tree 75 of 120
building tree 76 of 120
building tree 77 of 120
building tree 78

[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  5.7min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   11.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  4.7min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   10.6s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 120building tree 2 of 120
building tree 3 of 120

building tree 4 of 120
building tree 5 of 120
building tree 6 of 120
building tree 7 of 120
building tree 8 of 120
building tree 9 of 120
building tree 10 of 120
building tree 11 of 120
building tree 12 of 120
building tree 13 of 120
building tree 14 of 120
building tree 15 of 120
building tree 16 of 120
building tree 17 of 120
building tree 18 of 120
building tree 19 of 120
building tree 20 of 120
building tree 21 of 120
building tree 22 of 120
building tree 23 of 120
building tree 24 of 120
building tree 25 of 120
building tree 26 of 120
building tree 27 of 120
building tree 28 of 120
building tree 29 of 120
building tree 30 of 120
building tree 31 of 120
building tree 32 of 120
building tree 33 of 120
building tree 34 of 120
building tree 35 of 120
building tree 36 of 120


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.3min


building tree 37 of 120
building tree 38 of 120
building tree 39 of 120
building tree 40 of 120
building tree 41 of 120
building tree 42 of 120
building tree 43 of 120
building tree 44 of 120
building tree 45 of 120
building tree 46 of 120
building tree 47 of 120
building tree 48 of 120
building tree 49 of 120
building tree 50 of 120
building tree 51 of 120
building tree 52 of 120
building tree 53 of 120
building tree 54 of 120
building tree 55 of 120
building tree 56 of 120
building tree 57 of 120
building tree 58 of 120
building tree 59 of 120
building tree 60 of 120
building tree 61 of 120
building tree 62 of 120
building tree 63 of 120
building tree 64 of 120
building tree 65 of 120
building tree 66 of 120
building tree 67 of 120
building tree 68 of 120
building tree 69 of 120
building tree 70 of 120
building tree 71 of 120
building tree 72 of 120
building tree 73 of 120
building tree 74 of 120
building tree 75 of 120
building tree 76 of 120
building tree 77 of 120
building tree 78

[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  4.4min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   11.9s finished


In [10]:
all_predictions

,prediction
0,0.249287
1,0.924722
2,0.795912
3,0.572529
4,0.936332
...,...
1934169,0.752529
1934170,0.462410
1934171,0.864255
1934172,0.807485
